### Drowsiness Detector

- Uses dlib's frontal face detector to detect faces
- Calculates eye aspect ratio with the help of facial landmarks

In [4]:
import cv2
import dlib
from scipy.spatial import distance

# claculates eye aspect ratio
def calculate_EAR(eye):
    A = distance.euclidean(eye[1], eye[5])
    B = distance.euclidean(eye[2], eye[4])
    C = distance.euclidean(eye[0], eye[3])
    ratio = (A + B) / (2.0 * C)
    return ratio
    
    
cap = cv2.VideoCapture(0)

# use the frontal face detector
face_detector = dlib.get_frontal_face_detector()
dlib_facelandmark = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')

while True:
    success, frame = cap.read()
    
    # converting into gray img
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # detect faces
    faces = face_detector(gray)
    
    for face in faces:
        # detect landmarks in each face
        landmarks = dlib_facelandmark(gray, face)
        left_eye = []
        right_eye = []
        
        # drawing the left eye outline
        for n in range(36, 42):
            x = landmarks.part(n).x
            y = landmarks.part(n).y
            left_eye.append((x, y))
            
            next_point = n + 1            
            if n == 41:
                next_point = 36
            x2 = landmarks.part(next_point).x
            y2 = landmarks.part(next_point).y
            cv2.line(frame, (x, y), (x2, y2), (0, 255, 0), 1)
            
        # drawing the right eye outline
        for n in range(42, 48):
            x = landmarks.part(n).x
            y = landmarks.part(n).y
            right_eye.append((x, y))
            
            next_point = n + 1            
            if n == 47:
                next_point = 42
            x2 = landmarks.part(next_point).x
            y2 = landmarks.part(next_point).y
            cv2.line(frame, (x, y), (x2, y2), (0, 255, 0), 1)
            
        left_ear = calculate_EAR(left_eye)
        right_ear = calculate_EAR(right_eye)

        EAR = round((left_ear + right_ear) / 2, 2)
        
        if EAR < 0.2:
            cv2.putText(frame, 'DROWSY!', (10, 100), cv2.FONT_HERSHEY_SIMPLEX, 2, (0,0,255), 5)
            
    cv2.imshow('Drosiness Detector', frame)
    
    # close the frame on ESC key
    key = cv2.waitKey(1)
    if key == 27:
        break
    
# release and destroy all windows
cap.release()
cv2.destroyAllWindows()